In [1]:
%run ./etl_trusted_features.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 18:46:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/19 18:46:22 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
%pip install tensorflow --quiet
%pip install tensorflow_gnn --quiet
%pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.0 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

2023-03-19 18:47:53.983196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def getFirstEventFiles():
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(
        Bucket = 'sapient-bucket-trusted',
        Prefix = f'prod/graph/first_events')
    all_files = []
    for content in response.get('Contents', []):
        all_files.append(content['Key'])
        # print(content['Key'])
    files = [f"s3://sapient-bucket-trusted/" + f for f in all_files if 'parquet' in f]

    return files

In [5]:
# https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, test_split=0.1, val_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
columns = {"event_minute": tf.TensorSpec(tf.TensorShape([]), tf.int32),
           "event_day": tf.TensorSpec(tf.TensorShape([]), tf.int32), 
           "event_hour": tf.TensorSpec(tf.TensorShape([]), tf.int32)
                          }

In [7]:
# c_col = len(df.columns)

In [8]:
# import pyarrow.parquet as pq
# import s3fs
# s3 = s3fs.S3FileSystem()

In [9]:
# pandas_dataframe = pq.ParquetDataset('s3://sapient-bucket-trusted/prod/graph/first_events', filesystem=s3).read_pandas().to_pandas()

In [10]:
# TENSORFLOW CONFIGURATION
TRAIN_SIZE = 640
SHUFFLE_BUFFER = 500
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
tf.config.threading.set_inter_op_parallelism_threads(16)
pd.set_option('display.max_columns', None)

In [11]:
nonnull_ecar_cols = [
    'id', 'objectID','actorID','object','action','hostname', 'image_path', 
    'parent_image_path', 'new_path', 'file_path', 'malicious'
            ]

In [12]:
lb = LabelBinarizer()
mlb = MultiLabelBinarizer(sparse_output=True)

In [13]:
readFirstEvents().withColumn("malicious", when(col('malicious') == 1, "malicious")
                                            .otherwise("benign")) \
                .groupBy("malicious").count().show()

 6:48PM UTC on Mar 19, 2023 --- read time: 10.430203676223755 seconds ---


+---------+------+
|malicious| count|
+---------+------+
|   benign|981985|
|malicious|  3642|
+---------+------+



In [14]:
df = readFirstEvents().select(*nonnull_ecar_cols).cache() \
                    .withColumn("malicious", when(col('malicious') == 1, "malicious")
                                            .otherwise("benign")) \
                    .toPandas()

 6:48PM UTC on Mar 19, 2023 --- read time: 5.302300453186035 seconds ---


In [15]:
df.columns

Index(['id', 'objectID', 'actorID', 'object', 'action', 'hostname',
       'image_path', 'parent_image_path', 'new_path', 'file_path',
       'malicious'],
      dtype='object')

In [16]:
list_ds = tf.convert_to_tensor(df.fillna('unknown'))

2023-03-19 18:48:58.070616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 18:49:00.060273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 18:49:00.062136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-19 18:49:00.064273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [17]:
event_count = list_ds.shape[0]

In [18]:
numeric_dataset = tf.data.Dataset.from_tensor_slices(list_ds)

In [19]:
# shuffle the data
numeric_dataset = numeric_dataset.shuffle(event_count, reshuffle_each_iteration=False)

In [20]:
train_ds, test_ds, val_ds = get_dataset_partitions_tf(numeric_dataset, ds_size=event_count, train_split=0.8, test_split=0.1, val_split=0.1, 
                                                      shuffle=True, shuffle_size=10000)

In [21]:
print("Number of batches in train: ", train_ds.cardinality().numpy())
print("Number of batches in test: ", test_ds.cardinality().numpy())
print("Number of batches in val: ", val_ds.cardinality().numpy())

Number of batches in train:  788501
Number of batches in test:  98562
Number of batches in val:  98564


In [22]:
train_ds

<TakeDataset element_spec=TensorSpec(shape=(11,), dtype=tf.string, name=None)>

In [23]:
next(iter(train_ds))

<tf.Tensor: shape=(11,), dtype=string, numpy=
array([b'8a9b561f-c730-4cb0-9dd4-9d779f86ad23',
       b'c1939a02-704c-4c5d-8974-be980e9b7fd6',
       b'27413c5c-8b59-49e5-802b-583c184769e6', b'FLOW', b'INFO',
       b'SysClient0321.systemia.com', b'unknown', b'unknown', b'unknown',
       b'unknown', b'benign'], dtype=object)>

In [24]:
# convert the tensor object to a graph tensor object
gnn_ds = tf.identity(list_ds)

In [ ]:
# create graph with tensorflow gnn

In [ ]:
graph_schema = tfgnn.read_schema("./graph_schema.pbtxt")

In [ ]:
gtspec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

In [ ]:
dataset = numeric_dataset.map(tfgnn.keras.layers.ParseExample(gtspec))

In [ ]:
preproc_input_spec = dataset.element_spec

In [ ]:
# Define and apply the preprocessing model.
preproc_input = tf.keras.layers.Input(type_spec=preproc_input_spec)

In [ ]:
preproc_input